In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact
from scipy.stats import norm
import os
from collections import Counter
from scipy.stats import binomtest

hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

def parse_table(i):
    return [int(x) for x in i.replace("[", "").replace("]", "").split(",")]

d_abrev = {"LiangSteinNeuron":"FC exc. neur.", "FetalChondrocytes":"F chond.", "SertoliMale":"FG sertoli", "preGC_IIaFemale":"FG preGC IIa",\
          "NeuralFemale":"FG neur.", "FetalGonadImmuneFemale":"FG immune", "VIP":"AC VIP inh. neur.", "LiangSteinProgenitor":"FC prog.",\
          "AdultHeartVentricularCardiomyocyte":"AH cardiomyo.", "AdultLoopOfHenle":"AK loop of henle", "FetalBrainNeurGlioblast_CB_VZ":"FCB glioblast",\
         "AdultProximalTubule":"AK prox. tub.", "FetalLeydigMale":"FG leydig", "SST":"AC SST inh neur.", "KosoyRoussosControlMicroglia":"AC microglia",\
         "FetalBrainFloorPlate":"FB fl. plate", "FetalArterialECs":"FH endoth.", "ASCT":"AC astro.", "FetalBrainCOP":"FB COP",\
         "AMY":"AA neur.", "PVALB":"AC PVALB inh neur.", "ITL23":"AC L2-3 IT neur.", "FetalBrainNeurCB_GNP_IPC_1":"FB inter. prog.", "FetalBrainNeurDAergic":"FB DA neur.",\
          "OGC":"AC Oligo.", "D1Pu":"AP D1 inh neur.", "FetalBrainNeurSerotonergic":"FB 5-HT neur.", "FetalBrainNeurDRG_2":"FS DRG neur.",\
          "FetalHeartPericytes":"FH peri.", "FetalHeartEndocardium":"FH endocard.", "FetalHeartCardiacFibroblasts":"FH fibro.", "FetalBrainNeurPurkinje_6":"FCB Purk. inh neur.",\
          "AdultHeartSmoothMuscle":"AH smooth musc.", "FetalBrainRoofPlate":"FB ro. plate"}

In [ ]:
#Comparing across cutoffs
df = pd.DataFrame()

cutoff = 0.9
out = []
for cutoff in [0.6, 0.9]:
    for file in os.listdir("HAQERs_Results_Permute_FilterNewTEs"):
        if "FilterNewTEs" in file and "PhyloP447" not in file:
            vr = pd.read_csv("HAQERs_Results_Permute_FilterNewTEs/" + file, sep = "\t")
            vr = vr.drop(["TypeOfSampling"], axis = 1)
            vr.columns = list(vr.columns)[0:2] + list(vr.columns[3:]) + ["TypeOfSampling"]

            v2r = vr[vr["Proportion"] == cutoff]
            v2r = v2r[v2r["Top_20th"] == "All"]
            v2r = v2r[v2r["LFC_Cutoff"] == "NoLFCutoff"]
            v2r = v2r[v2r["Distance_bin"] == "InHAQERs"]
            v2r = v2r[v2r["Iteration"] == "Real"]
            v2r["Cell type"] = np.repeat(d_abrev[file.replace("HAQERs_abslogfc_", "").replace(".txt.gz", "").replace("FilterNewTEs_", "")], v2r.shape[0])
            out.append([d_abrev[file.replace("HAQERs_abslogfc_", "").replace(".txt.gz", "").replace("FilterNewTEs_", "")], cutoff, float(v2r["alpha"])])
df = pd.DataFrame(out)
df

C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:8: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  vr = pd.read_csv("HAQERs_Results_Permute_FilterNewTEs/" + file, sep = "\t")
C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  out.append([d_abrev[file.replace("HAQERs_abslogfc_", "").replace(".txt.gz", "").replace("FilterNewTEs_", "")], cutoff, float(v2r["alpha"])])
C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:8: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  vr = pd.read_csv("HAQERs_Results_Permute_FilterNewTEs/" + file, sep = "\t")
C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a

C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:8: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  vr = pd.read_csv("HAQERs_Results_Permute_FilterNewTEs/" + file, sep = "\t")
C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  out.append([d_abrev[file.replace("HAQERs_abslogfc_", "").replace(".txt.gz", "").replace("FilterNewTEs_", "")], cutoff, float(v2r["alpha"])])
C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:8: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  vr = pd.read_csv("HAQERs_Results_Permute_FilterNewTEs/" + file, sep = "\t")
C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a

C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:8: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  vr = pd.read_csv("HAQERs_Results_Permute_FilterNewTEs/" + file, sep = "\t")
C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  out.append([d_abrev[file.replace("HAQERs_abslogfc_", "").replace(".txt.gz", "").replace("FilterNewTEs_", "")], cutoff, float(v2r["alpha"])])
C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:8: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  vr = pd.read_csv("HAQERs_Results_Permute_FilterNewTEs/" + file, sep = "\t")
C:\Users\astar\AppData\Local\Temp\ipykernel_25988\2509839009.py:18: FutureWarning: Calling float on a single element Series is deprecated and will raise a

,0,1,2
0,AH smooth musc.,0.6,0.003618
1,AH cardiomyo.,0.6,0.103652
2,AK loop of henle,0.6,0.043278
3,AK prox. tub.,0.6,-0.016655
4,AA neur.,0.6,-0.068634
...,...,...,...
63,FG preGC IIa,0.9,0.142841
64,AC PVALB inh neur.,0.9,0.116600
65,FG sertoli,0.9,0.081071
66,AC SST inh neur.,0.9,0.078436


In [ ]:
from scipy.stats import pearsonr
pearsonr(df[df[1] == 0.9][2], df[df[1] == 0.6][2])

In [ ]:
fig, ax = plt.subplots(figsize = (8, 6))
sns.regplot(x = df[df[1] == 0.9][2], y = df[df[1] == 0.6][2])
plt.title("HAQERs", size = 20)
plt.xlabel("$\\alpha_{CA}$, 90th percentile", size = 18)
plt.ylabel("$\\alpha_{CA}$, 60th percentile", size = 18)

In [ ]:
#Read in data across cell types for HARs or HAQERs
df = pd.DataFrame()

cutoff = 0.9
for file in os.listdir("HARs_Results_Permute_FilterNewTEs"):
    if "FilterNewTEs" in file and "PhyloP447" not in file:
        vr = pd.read_csv("HARs_Results_Permute_FilterNewTEs/" + file, sep = "\t")
        vr = vr.drop(["TypeOfSampling"], axis = 1)
        vr.columns = list(vr.columns)[0:2] + list(vr.columns[3:]) + ["TypeOfSampling"]

        v2r = vr[vr["Proportion"] == cutoff]
        v2r = v2r[v2r["Top_20th"] == "All"]
        v2r = v2r[v2r["LFC_Cutoff"] == "NoLFCutoff"]
        v2r = v2r[v2r["Distance_bin"] == "InHARs"]
        v2r["Cell type"] = np.repeat(d_abrev[file.replace("HARs_abslogfc_", "").replace(".txt.gz", "").replace("FilterNewTEs_", "")], v2r.shape[0])
        df = pd.concat([df, v2r])
df

In [ ]:
#Compute medians and make plots
medians = []
ci = []
palette = {}
for ct in np.unique(df["Cell type"]):
    dff = df[df["Cell type"].isin([ct])]
    dff = dff.sort_values("alpha")
    ci.append([ct, list(dff["alpha"])[250], list(dff["alpha"])[9750]])
    medians.append([ct, np.median(dff["alpha"])])
    
df_med = pd.DataFrame(medians).sort_values(1, ascending = False)
order = list(df_med[0])

df_ci = pd.DataFrame(ci)
df_ci.columns = ["CellType", "Lower95", "Upper95"]
df_ci = df_ci.sort_values("Lower95", ascending = False)

for index, row in df_ci.iterrows():
    if row["Lower95"] > 0:
        palette[row["CellType"]] = "#F42FF5"
    else:
        palette[row["CellType"]] = "#40A94D"

fig, ax = plt.subplots(figsize = (10, 4))
sns.boxplot(data = df, y = "alpha", x = "Cell type", order = order, palette = palette)
plt.ylim(-1.25, 0.6)
plt.ylabel("$\\alpha_{CA}$", size = 18)
plt.xlabel("Cell type", size = 18)

plt.xticks(rotation = 90, size = 12)
plt.title("Positive selection in HAQERs across cell types", size = 20)
plt.yticks(size = 16)
df_ci

In [ ]:
v = pd.read_csv("Fixed_LiangSteinNeuron.txt.gz", sep = "\t")
vv = pd.read_csv("Poly_MAF0.25_LiangSteinNeuron.txt.gz", sep = "\t")


try:
    vv = add_unfold(vv)
    vv_ref = vv[vv["Human ref"] == vv["Chimp ref"]]
    vv_alt = vv[vv["Human alt"] == vv["Chimp ref"]]
    vv_ref["fixed logfc"] = -vv_ref["logfc"].astype(float)
    vv_alt["fixed logfc"] = vv_alt["logfc"].astype(float)
    vv = pd.concat([vv_ref, vv_alt])
except:
    pass

te_blacklist = pd.read_csv("BlacklistTE_Variants.txt", sep = "\t")
    
v = v[~v["Position"].isin(te_blacklist["Position"])]
vv = vv[~vv["Position"].isin(te_blacklist["Position"])]



In [ ]:
harf = pd.read_csv("hCONDELs_HAQERs_HARs/HumChp_NC_Final_Rmdup_CREs_NoHLA_HARs.bed", header = None, sep = "\t")
harp = pd.read_csv("hCONDELs_HAQERs_HARs/HumPoly_NC_Final_CREs_NoHLA_HARs.bed", header = None, sep = "\t")

harf

#Read in the HAR data from the CRIPSRi paper
o = open("HAR_CRISPRi.txt")
pos = []
for line in o:
    p = line.split("\t")[2].replace('"', '').replace(",", "")
    try:
        pos = pos + [p.split(":")[0] + ":" + str(_) for _ in list(range(int(p.split(":")[1].split("-")[0]) - 5, int(p.split(":")[1].split("-")[1]) + 5))]
    except:
        print(p)
pos

#Look for interesting ones
v[v["Position"].isin(pos)].sort_values("logfc")

In [ ]:
#Create volcano plots for enrichments
df = pd.read_csv("greatExportAll_HAR_AbsG0.1_MousePhenotypeSingleKO.tsv", sep = "\t", skiprows=3).dropna()
out = []
for index, row in df.iterrows():
    if len(row["BgGeneNames"].split(",")) >= 5:
        out.append(row)
df = pd.DataFrame(out)

dff = df.copy()
dff["-Log$_{10}$(FDR)"] = -np.log10(dff["HyperFdrQ"])

k = []
for index, row in dff.iterrows():
    if row["HyperFdrQ"] < 0.05:
        k.append("FDR < 0.05")
    else:
        k.append("Not significant")
dff['Significance'] = k
fig, ax = plt.subplots(figsize=(8,6))
sns.scatterplot(data = dff, x = "RegionFoldEnrich", y = "-Log$_{10}$(FDR)", hue = "Significance", palette = {"FDR < 0.05":"red", "Not significant":"grey"})
plt.title("GREAT enrichments, mouse KO phenotypes in HARs", size = 20)
plt.ylabel("-Log$_{10}$(FDR)", size = 18)
plt.xlabel("Hypergeometric enrichment", size = 18)
plt.xticks(size = 14)
plt.yticks(size = 14)
plt.legend(fontsize = 14)



In [ ]:
#Organize genes for HAR biorender figure using SYNGO information
local = {"Nucleus":["ARX", "CDH7", "EBF2", "EMX2", "ETV1", "NEUROD1", "SOX2"], \
         "Presynapse":["CACNG2", "CNTNAP2", "GAD1", "GLRA1", "KCNMA1", "KCTD12", "NFASC", "NRG1", "SLC6A9", "TPH2"], \
         "Postsynapse":["ASIC2", "CACNG2", "CNTNAP2", "DPP6", "ELFN1", "GAD1", "GLRA1", "GPHN", "GRID2", "GRIK1", "GRIK2", "GRIN3A", "KCTD12", "NRG1", "SLC6A9"], \
         "Membrane":["GRXCR1", "IL6ST", "KCNMB2", "SSTR1", "TNR", "TRPC4"], "Cytoplasm":["LPIN1"]}

for i in local["Membrane"]:
    print(i)

In [ ]:
syn = pd.read_csv("syngo_annotations.csv")
syn[syn["hgnc_symbol"].isin(df[df["Desc"] == "abnormal nervous system electrophysiology"]["FgGeneNames"][1].split(","))].tail(50)

In [ ]:
df[df["Desc"] == "abnormal nervous system electrophysiology"]["FgGeneNames"][1].split(",")

In [ ]:
#Create inputs for HAR enrichment testing with GREAT
v2_fore = v2[v2["abs logfc"] > 0.1]

v2_fore["Chrom"] = [x.split(":")[0] for x in v2_fore["Position.1"]]
v2_fore["Pos1"] = [int(x.split(":")[1]) for x in v2_fore["Position.1"]]
v2_fore["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2_fore["Position.1"]]
v2_fore = v2_fore[["Chrom", "Pos1", "Pos2"]]

#Need to change 
v2_fore.to_csv("HAR_AbsG0.1_Fixed_LiangSteinNeuron_FilterNewTEs_All.bed", sep = "\t", header = None, index = None)

v2["Chrom"] = [x.split(":")[0] for x in v2["Position.1"]]
v2["Pos1"] = [int(x.split(":")[1]) for x in v2["Position.1"]]
v2["Pos2"] = [int(x.split(":")[1]) + 1 for x in v2["Position.1"]]

#Set to 0.05 for MEF2A
#Change 0.025 to 0.1 for NFIB and other TFs with large effects on CA
v2_back = pd.concat([v2_fore, v2[v2["abs logfc"] < 0.025]])
v2_back = v2_back[["Chrom", "Pos1", "Pos2"]]
v2_back.to_csv("HAR_AbsL0.025_G0.1_Fixed_Background_New_LiangSteinNeuron_FilterNewTEs_All.bed", sep = "\t", header = None, index = None)
